## Mock data taken from Kaggle
#### There are two data files: GME_stock.csv which contains the stock price the other file is also from Kaggle which contains "positive" and "negative" sentiments. I thought this would be a good data to do some practice run.

In [27]:
# Initial imports
import pandas as pd
from sklearn.cluster import KMeans
import plotly.express as px
import hvplot.pandas
import numpy as np

In [28]:
# Load gamestop data
file_path = "Mock_data/GME_stock.csv"
df_GME_price = pd.read_csv(file_path)
df_GME_price.head(10)

,date,open_price,high_price,low_price,close_price,volume,adjclose_price
0,2021-01-28,265.000000,483.000000,112.250000,193.600006,58815800.0,193.600006
1,2021-01-27,354.829987,380.000000,249.000000,347.510010,93396700.0,347.510010
2,2021-01-26,88.559998,150.000000,80.199997,147.979996,178588000.0,147.979996
3,2021-01-25,96.730003,159.179993,61.130001,76.790001,177874000.0,76.790001
4,2021-01-22,42.590000,76.760002,42.320000,65.010002,196784300.0,65.010002
5,2021-01-21,39.230000,44.750000,37.000000,43.029999,57079800.0,43.029999
6,2021-01-20,37.369999,41.189999,36.060001,39.119999,33471800.0,39.119999
7,2021-01-19,41.549999,45.520000,36.639999,39.360001,74721900.0,39.360001
8,2021-01-15,38.490002,40.750000,34.009998,35.500000,46752200.0,35.500000
9,2021-01-14,38.090000,43.060001,33.049999,39.910000,93717400.0,39.910000


In [29]:
# Check data types
df_GME_price.dtypes

date               object
open_price        float64
high_price        float64
low_price         float64
close_price       float64
volume            float64
adjclose_price    float64
dtype: object

In [30]:
# Load Reddit data
file_path1 = "Mock_data/GME sentiment analysis reddit.csv"
df_reddit = pd.read_csv(file_path1)
df_reddit.head()

,Unnamed: 0,title,body,timestamp,subjectivity,polarity,analysis
0,0,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,NaN,28-01-2021 21:28,0.480303,-0.238826,Negative
1,1,"Not to distract from GME, just thought our AMC...",NaN,28-01-2021 21:26,0.250000,0.250000,Positive
2,2,Currently Holding AMC and NOK - Is it retarded...,NaN,28-01-2021 21:19,0.600000,-0.400000,Negative
3,3,GME Premarket ???�?� Musk approved ???????????...,NaN,28-01-2021 21:17,0.000000,0.000000,Neutral
4,4,"Once you're done with GME - $AG and $SLV, the ...",You guys are champs. GME... who would have tho...,28-01-2021 21:17,0.300000,0.000000,Neutral


In [31]:
# Check data types
df_reddit.dtypes

Unnamed: 0        int64
title            object
body             object
timestamp        object
subjectivity    float64
polarity        float64
analysis         object
dtype: object

### Preprocess the data

In [32]:
# Remove the timestamp from the date of the reddit data
df_reddit["timestamp"] = pd.to_datetime(df_reddit['timestamp'], dayfirst=True)
df_reddit["new_date_column"] = df_reddit["timestamp"].dt.date
df_reddit.head()

,Unnamed: 0,title,body,timestamp,subjectivity,polarity,analysis,new_date_column
0,0,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,NaN,2021-01-28 21:28:00,0.480303,-0.238826,Negative,2021-01-28
1,1,"Not to distract from GME, just thought our AMC...",NaN,2021-01-28 21:26:00,0.250000,0.250000,Positive,2021-01-28
2,2,Currently Holding AMC and NOK - Is it retarded...,NaN,2021-01-28 21:19:00,0.600000,-0.400000,Negative,2021-01-28
3,3,GME Premarket ???�?� Musk approved ???????????...,NaN,2021-01-28 21:17:00,0.000000,0.000000,Neutral,2021-01-28
4,4,"Once you're done with GME - $AG and $SLV, the ...",You guys are champs. GME... who would have tho...,2021-01-28 21:17:00,0.300000,0.000000,Neutral,2021-01-28


In [33]:
df_reddit["new_date_column"].value_counts()

2021-01-29    2940
2021-02-06     619
2021-02-05     550
2021-02-03     536
2021-02-02     310
              ... 
2021-04-03       3
2021-02-15       2
2021-04-04       2
2021-04-05       2
2021-02-16       2
Name: new_date_column, Length: 68, dtype: int64

In [34]:
# Drop columns not needed. 
reddit_clean_df = df_reddit.drop(columns=["Unnamed: 0", "title", "body", "timestamp"], axis=True)
reddit_clean_df.head()

,subjectivity,polarity,analysis,new_date_column
0,0.480303,-0.238826,Negative,2021-01-28
1,0.250000,0.250000,Positive,2021-01-28
2,0.600000,-0.400000,Negative,2021-01-28
3,0.000000,0.000000,Neutral,2021-01-28
4,0.300000,0.000000,Neutral,2021-01-28


In [35]:
reddit_clean_df["date"] = reddit_clean_df["new_date_column"]
reddit_clean_df1=reddit_clean_df.drop(columns=["new_date_column","analysis"])
reddit_clean_df1.head()

,subjectivity,polarity,date
0,0.480303,-0.238826,2021-01-28
1,0.250000,0.250000,2021-01-28
2,0.600000,-0.400000,2021-01-28
3,0.000000,0.000000,2021-01-28
4,0.300000,0.000000,2021-01-28


In [37]:
# # Remove the timestamp from the date of the reddit data
# reddit_clean_df["timestamp"] = pd.to_datetime(reddit_clean_df['timestamp'])
# reddit_clean_df["new_date_column"] = reddit_clean_df["timestamp"].dt.date
# reddit_clean_df.head()
reddit_clean_df2 = reddit_clean_df1.groupby('date')['subjectivity','polarity'].sum()
reddit_clean_df2['date']=reddit_clean_df2.index
reddit_clean_df2 = reddit_clean_df2.reset_index(drop=True)
reddit_clean_df2

C:\Users\dfand\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


,subjectivity,polarity,date
0,65.905308,8.809233,2021-01-28
1,734.038456,72.789893,2021-01-29
2,79.178550,5.050014,2021-01-30
3,75.969506,9.969900,2021-01-31
4,66.381158,5.184502,2021-02-01
...,...,...,...
63,1.722222,-0.288889,2021-04-01
64,3.094444,0.473394,2021-04-02
65,0.854167,0.715625,2021-04-03
66,0.283333,0.250000,2021-04-04


In [42]:
reddit_clean_df2.to_csv("Mock_data/GME_stock_Cleaned.csv")
reddit_clean_df2.dtypes

subjectivity    float64
polarity        float64
date             object
dtype: object

#### Practice checking data

In [39]:
# Look at the GME stock data to get some idea how it looks like
# df_GME_price.hvplot.scatter(x="open_price", y="close_price", by="date")
df_GME_price.hvplot.scatter(x="open_price", y="close_price", hover_cols=["date", "volume"])

:Scatter   [open_price]   (close_price,date,volume)

In [49]:
# Join the two dataframes with the Reddit data in the left and GME stock price on the right
reddit_gme_df = reddit_clean_df2.merge(df_GME_price, on='date', how='left')
reddit_gme_df.head()

,subjectivity,polarity,date,open_price,high_price,low_price,close_price,volume,adjclose_price
0,65.905308,8.809233,2021-01-28,NaN,NaN,NaN,NaN,NaN,NaN
1,734.038456,72.789893,2021-01-29,NaN,NaN,NaN,NaN,NaN,NaN
2,79.178550,5.050014,2021-01-30,NaN,NaN,NaN,NaN,NaN,NaN
3,75.969506,9.969900,2021-01-31,NaN,NaN,NaN,NaN,NaN,NaN
4,66.381158,5.184502,2021-02-01,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
reddit_gme_df.to_csv("Mock_data/Combined_GME_Data.csv")

In [24]:
# Look at the GME stock data to get some idea how it looks like
# df_GME_price.hvplot.scatter(x="open_price", y="close_price", by="date")
df_GME_price.hvplot.scatter(x="date", y="close_price",  rot=90)

:Scatter   [date]   (close_price)

### Next step is to think about preparing the data for machine learning.